In [79]:
import pandas as pd

In [127]:
j1_mdg = pd.read_excel('../sync/c-before-sync/mdg/j1.xlsx', dtype=str)
j1_ecc = pd.read_excel('../sync/c-before-sync/ecc/j1.xlsx', dtype=str)

In [128]:
print(len(j1_ecc.columns))
print(len(j1_mdg.columns))

16
14


In [129]:
cols_mdg = set(j1_mdg.columns.str.lower())
cols_ecc = set(j1_ecc.columns.str.lower())

In [130]:
only_in_mdg = sorted(cols_mdg - cols_ecc)
only_in_ecc = sorted(cols_ecc - cols_mdg)

In [131]:
df_only_mdg = pd.DataFrame({'In MDG not in ECC': only_in_mdg})
df_only_ecc = pd.DataFrame({'In ECC not in MDG': only_in_ecc})

In [132]:
print(df_only_ecc)
print(df_only_mdg)

      In ECC not in MDG
0        aadhaar number
1  gst tds registration
Empty DataFrame
Columns: [In MDG not in ECC]
Index: []


In [133]:
with pd.ExcelWriter('../sync/c-before-sync/column_mismatch_j1lsx', engine='openpyxl') as writer:
    df_only_mdg.to_excel(writer, sheet_name='In MDG', index=False)
    df_only_ecc.to_excel(writer, sheet_name='In ECC', index=False)

In [134]:
n_cols_ecc = list(set(j1_ecc.columns) - (set(j1_ecc.columns) - set(j1_mdg.columns)))
n_cols_mdg = list(set(j1_mdg.columns) - (set(j1_mdg.columns) - set(j1_ecc.columns)))

In [135]:
# n_cols_ecc.append('Supplier')
# n_cols_mdg.append('Vendor')

In [136]:
j1_ecc = j1_ecc[n_cols_ecc]
j1_mdg = j1_mdg[n_cols_mdg]

In [137]:
print(j1_ecc.columns)
print(j1_mdg.columns)

Index(['ECC No.', 'Excise Division', 'Excise Registration No.',
       'Excise Commissionerate', 'PAN Reference Number',
       'Exc.Tax Ind. Custom.', 'Excise Range',
       'Service Tax Registration Number', 'Permanent account number',
       'LST number', 'CST number', 'Changed on', 'Changed By the user',
       'Customer'],
      dtype='object')
Index(['ECC No.', 'Excise Division', 'Excise Range', 'PAN Reference Number',
       'Service Tax Registration Number', 'Excise Registration No.',
       'Excise Commissionerate', 'Permanent account number', 'LST number',
       'CST number', 'Changed on', 'Changed By the user',
       'Exc.Tax Ind. Custom.', 'Customer'],
      dtype='object')


In [138]:
j1_ecc['key'] = j1_ecc.apply(
    lambda row: 
        str(row['Customer']),
        # + str(row['Contact Person']),
        # + str(row['Distribution Channel'])
        # + str(row['Division']),
        # + str(row['Sequence Number']), 
        axis=1
)

In [139]:
j1_mdg['key'] = j1_mdg.apply(
    lambda row: 
        str(row['Customer']),
        # + str(row['Contact Person']),
        # + str(row['Distribution Channel'])
        # + str(row['Division']),
        # + str(row['Sequence Number']),
    axis=1
)

In [140]:
print(len(j1_ecc))
print(len(j1_mdg))

2312
2134


In [141]:
j1_ecc.set_index('key', inplace=True)
j1_mdg.set_index('key', inplace=True)

In [143]:
a_j1_mdg, a_j1_ecc = j1_mdg.align(j1_ecc, join='outer')
# a_j1_mdg, a_j1_ecc = j1_mdg, j1_ecc


In [144]:
print(len(a_j1_ecc))
print(len(a_j1_mdg))

2323
2323


In [148]:
diffs = a_j1_mdg.compare(
    a_j1_ecc,
    # keep_equal=True,      # show all cells
    keep_equal=False,      # only show changed cells
    keep_shape=True,       # retain full shape if you want blanks where no change
    result_names=('MDG','ECC')
)

In [149]:
diffs.columns = [f"{col[0]} ({col[1]})" for col in diffs.columns]

In [150]:
# diffs.to_excel('../sync/c-before-sync/j1_allData_changes.xlsx')
diffs.to_excel('../sync/c-before-sync/j1_changes.xlsx')

In [73]:
# Ignore

In [ ]:
f = pd.read_excel('../j1Data_changes.xlsx', sheet_name='Sheet3')

In [148]:
f.head()

,Name,email
0,3M Electro & Communication,pbhutani@mmm.com
1,3M Electro & Communication,srjanak@mmm.com
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM
4,AAALAC INTERNATIONAL,msprouse@aaalac.org


In [153]:
def func(df):
    n = df['Name'].split()
    
    for i in n:
        if i.lower() in df['email'].lower():
            return True
    return False

In [154]:
f['new'] = f.apply(lambda row: func(row), axis=1)

In [155]:
f

,Name,email,new
0,3M Electro & Communication,pbhutani@mmm.com,False
1,3M Electro & Communication,srjanak@mmm.com,False
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM,True
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM,True
4,AAALAC INTERNATIONAL,msprouse@aaalac.org,True
...,...,...,...
2828,ZELLE BIOTECHNOLOGY PVT LTD,accounts@zellebiotech.com,True
2829,ZELLE BIOTECHNOLOGY PVT LTD,orders@zellebiotech.com,True
2830,ZELLE BIOTECHNOLOGY PVT LTD,service@zellebiotech.com,True
2831,ZoomInfo Technologies LLC,ar@zoominfo.com,True


In [156]:
f.to_csv('email.csv')

In [ ]:
j1cc.columns

Index(['Address Number', 'from', 'Address Version', 'To', 'Title', 'Name',
       'Name 2', 'Name 3', 'Name 4', 'Converted name (with form of address)',
       'c/o name', 'City', 'District', 'City Code', 'District.1',
       'City (Diff. from Postal City)', 'City Code.1', 'Test stat./City file',
       'Reg. Struct. Grp.', 'Postal Code', 'PO Box Postal Code',
       'Company Postal Code', 'Postl Code Extension', 'Postl Code Extension.1',
       'Postl Code Extension.2', 'PO Box', 'Undeliverable', 'PO Box w/o No.',
       'PO Box City', 'City Code.2', 'PO Box Region', 'PO box country',
       'Delivery District', 'Transportation zone', 'Street', 'Undeliverable.1',
       'Street Code', 'Street Abbreviation', 'House Number', 'Supplement',
       'House Number Range', 'Street 2', 'Street 3', 'Street 4', 'Street 5',
       'Building Code', 'Floor', 'Room Number', 'Country Key', 'Language Key',
       'Region', 'Address group', 'Flag: There are more address group assig',
       'personal a

In [ ]:
j1.apply(lambda row: '&' in str(row['Street']), axis=1).value_counts()

False    2944
True       78
Name: count, dtype: int64